In [2]:

import torch
from transformers import T5EncoderModel, T5Tokenizer
import re
import numpy as np
import gc
from tqdm import tqdm



tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Loading model...")
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
print("Model loaded successfully.")



In [1]:
from transformers import T5Tokenizer, T5EncoderModel

# Specify the directory where you want to save the model
local_directory = "."

# Download and save the tokenizer
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False)
tokenizer.save_pretrained(local_directory)

# Download and save the model
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
model.save_pretrained(local_directory)

print("Model and tokenizer saved locally")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Model and tokenizer saved locally


In [10]:

def generate_embedding(seq):
    # print(seq)
    # print(len(seq)) 
    # Ensure the model and tokenizer are on the same device (preferably GPU)
    model.eval()  # Set the model to evaluation mode
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(DEVICE)

    # Tokenize the protein sequence
    inputs = tokenizer(seq, return_tensors="pt", truncation=True, max_length=512)
    inputs = {key: tensor.to(DEVICE) for key, tensor in inputs.items()}

    # print(inputs)
    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    return outputs.last_hidden_state[:,1:, :].cpu().numpy()


def main():
    # take input from the user
    seq = input("Enter the protein sequence: ")
    seq = re.findall(r'[A-Z]', seq)
    seq = ' '.join(seq)
    # print(len(seq))
    if(len(seq) > 1024):
        print("Sequence length is greater than 1024. Truncating the sequence.")
        seq = seq[:1024]

    seq = ' '.join(seq)

    embedding = generate_embedding(seq)
    # print(embedding)
    print(embedding.shape)

main()

(1, 11, 1024)
